<a href="https://colab.research.google.com/github/dwdsuh/OMOS/blob/master/Advanced_DL_practices_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-concept of inception module & residual connection

#1. Intro to the functional API

In [0]:
from keras import Input, layers

input_tensor=Input(shape=(32,))
dense=layers.Dense(32, activation='relu')
output_tensor=dense(input_tensor)

## 1) Comparing Sequential model and API function

In [0]:
from keras.models import Sequential, Model

seq_model=Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor=Input(shape=(64,))
x=layers.Dense(32, activation='relu')(input_tensor)
x=layers.Dense(32, activation='relu')(x)
output_tensor=layers.Dense(10, activation='softmax')(x)
model=Model(input_tensor, output_tensor)

seq_model.summary()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 

##2) Multi-input models

In [0]:
from keras.models import Model
from keras import layers
from keras import Input

text_voc_size=10000
quest_voc_size=10000
answer_voc_size=500

text_input=Input(shape=(None,), dtype='int32', name='text')
embedded_text=layers.Embedding(text_voc_size, 64)(text_input)
encoded_text=layers.LSTM(32)(embedded_text)

quest_input=Input(shape=(None,), dtype='int32', name='question')
embedded_question=layers.Embedding(quest_voc_size, 32)(quest_input)
encoded_question=layers.LSTM(16)(embedded_question)


concatenated=layers.concatenate([encoded_text, encoded_question], axis=-1)
answer=layers.Dense(answer_voc_size, activation='softmax')(concatenated)

model=Model([text_input, quest_input], answer)
model.summary()

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS

In [0]:
import numpy as np
num_samples=1000
max_len=100

text=np.random.randint(1, text_voc_size, size=(num_samples, max_len))
question=np.random.randint(1, quest_voc_size, size=(num_samples, max_len))
answers=np.random.randint(0,1, size=(num_samples, answer_voc_size))
model.fit([text, question], 
          answers, 
          epochs=10, 
          batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 1.0000e-03
Epoch 1

##3) Multi-ouput model

In [0]:
from keras.models import Model
from keras import layers
from keras import Input

voc_size=50000
num_income_groups=10

posts_input=Input(shape=(None,), dtype='int32', name='posts')
embedded_posts=layers.Embedding(voc_size, 256)(posts_input)
x=layers.Conv1D(126, 5, activation='relu')(embedded_posts)
x=layers.MaxPooling1D(5)(x)
x=layers.Conv1D(126, 5, activation='relu')(embedded_posts)
x=layers.Conv1D(126, 5, activation='relu')(embedded_posts)
x=layers.MaxPooling1D(5)(x)
x=layers.MaxPooling1D(5)(x)
x=layers.GlobalMaxPooling1D()(x)

age_pred=layers.Dense(1, name='age')(x)
income_pred=layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_pred=layers.Dense(1, activation='sigmoid', name='gender')(x)

model=Model(posts_input, [age_pred, income_pred, gender_pred])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, None, 126)    161406      embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_3 (MaxPooling1D)  (None, None, 126)    0           conv1d_5[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [0]:
model.compile(optimizer='rmsprop',
             loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])


##4) Directed acyclic graphs of layers

In [0]:
#Inception Module

from keras import layers

x=Input(shape=(None,None,3), dtype='float32', name='x')

branch_a=layers.Conv2D(128, 1, activation='relu', strides=2)(x)
branch_b=layers.Conv2D(128, 1, activation='relu')(x)
branch_b=layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c=layers.Conv2D(128,1,activation='relu')(x)
branch_c=layers.Conv2D(128,3, activation='relu')(branch_c)

concatenate=layers.concatenate([branch_a, branch_b, branch_c], axis=-1)
#output=layers.Dense(1, activation='sigmoid')(concatenate)

model=Model(x, concatenate)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, None, 1 512         x[0][0]                          
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, None, None, 1 512         x[0][0]                          
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 512         x[0][0]                          
__________________________________________________________________________________________________
conv2d_3 (

In [0]:
## Residual Connection
#to overcome vanishing gradients and representational bottlenecks

x=Input(shape=(None, None,1), dtype='float32', name='x')#some 4D input tensor
y=layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y=layers.Conv2D(128, 3, activation='relu',padding='same')(y)
y=layers.add([y,x])
output=layers.GlobalMaxPooling2D()(y)

model=Model(x,output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, None, None, 1 1280        x[0][0]                          
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, None, None, 1 147584      conv2d_6[0][0]                   
__________________________________________________________________________________________________
add_1 (Add)                     (None, None, None, 1 0           conv2d_7[0][0]                   
                                                                 x[0][0]                          
__________

## 5) Layer Weight Sharing

In [0]:
#psuedo

lstm=layers.LSTM(32)

left_input=Input(shape=(None, 128))
left_output=lstm(left_input)

right_input=Input(shape=(None, 128))
right_output=lstm(right_output)


merged=layers.concatenate([left_output, right_output], axis=-1)
pred=layers.Dense(1, activation='sigmoid')(merged)

model=Model([left_input, right_input], pred)
model.fit([left_data, right_data], targets)

NameError: ignored

#Keras callbacks and TensorBoard
for inspecting and monitoring deep-learning models

##1) using callbacks to act on a model during training

- model checkpointing
- Early stopping
- Dynamically adjusting the value of certain parameters during training
- Logging training and validation metics duing training, or visualinzing the representations learned by the model as they're updated

### model checkpoint

In [0]:
#with a pre-defined model

import keras

callbacks_list=[keras.callbacks.EarlyStopping(monitor='acc',
                                              patience=1,),
               keras.callbacks.ModelCheckpoint(filepath='my_model.h5',
                                               monitor='val_loss', 
                                               save_best_only=1,)]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metics=['acc'])

model.fit(x,y,
         epochs=10,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))



###ReduceLROnPlateau
--> reduce the learning rate on plateau

In [0]:
callbacks_list=[keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                factor=0.1,#divide LR by 10 when triggered
                                                patience=10,)]

model.fit(x,y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val,y_val))


###Making your own callback

In [0]:
import keras 
import numpy as np

class ActivaitonLogger(keras.callbacks.Callback):
  def set_model(self, model):
    self.model=model
    layer_outputs=[layer.output for layer in model.layeers]
    self.activations_model=keras.models.Model(model.input,
                                             layer_outputs)
  def on_epoch_end(self, epoch, logs=None):
    if self.validation_data is None:
      raise RuntimeError('Requires validation_data.')
    validation_sample=self.validation_data[0][0:1]
    activations=self.activations_model.predict(validation_sample)
    f=open('activation_at_epoch_'+str(epoch)+'.npz','w')
    np.savez(f, activations)
    f.close()


##2) Intro to TensorBoard:
the tensorflow visualization framework

- visually monitoring metrics during training
- visualizing your model architecture
- visualizing histograms of activations and gredients
- Exploring embeddings in 3D

In [0]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features=2000
max_len=500

(x_train, y_train), (x_test, y_test)=imdb.load_data(num_words=max_features)
x_train=sequence.pad_sequences(x_train, maxlen=max_len)
x_test=sequence.pad_sequences(x_test, maxlen=max_len)

model=keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32,7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='model1.png')

In [0]:
!mkdir my_log_dir

In [0]:
##don't understand why it's not workin'

callbacks=[keras.callbacks.TensorBoard(log_dir='my_log_dir',
                                      histogram_freq=1,
                                      embeddings_freq=1,
                                      )]

history=model.fit(x_train, y_train,
                 epochs=20,
                 batch_size=128,
                 validation_split=0.2,
                 callbacks=callbacks)

#Getting the most out of your models

##1) Advanced architecture patterns

In [0]:
##Batch Normalization

conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(layers,BatchNormalization())


Batch renomalization



In [0]:
##Depthwise Separable Convolution

#sample

from keras.models import Sequential, Model
from keras import layers

height=64
width=64
channels=3
num_classes=10

model=Sequential()
model.add(layers.SeparableConv2D(32,3,activation='relu',
                                input_shape=(height,width, channels,)))
model.add(layers.MaxPooling2D(2))
model.add(layers.SeparableConv2D(64,3,activation='relu'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_2 (Separabl (None, 62, 62, 32)        155       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 29, 29, 64)        2400      
Total params: 2,555
Trainable params: 2,555
Non-trainable params: 0
_________________________________________________________________


##2) Hyperparameter optimization

library: Hyperopt, Hyperas

it is still young and immature